In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.utils import prepare_fashion, prepare_mnist

from libs.constants import MODELS_FOLDER

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [7]:
# define dataset and model architecture
# parser = argparse.ArgumentParser()
# parser.add_argument('dataset')
# parser.add_argument('model_arch')
# args= parser.parse_args()

# dataset = args.dataset
# model_arch = args.model_arch

dataset = "fashion"
model_arch = "lenetA"

In [8]:
# prepare data
if dataset == "fashion":
    (x_train, y_train), (x_test, y_test) = prepare_fashion()
elif dataset == "mnist":
    (x_train, y_train), (x_test, y_test) = prepare_mnist()
else:
    print("Invalid Dataset or Dataset not found")
        

In [9]:
seed = model_seeds[0]
# load model
model_instance = dataset + "--" + model_arch + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model_arch)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model = models.load_model(model_file)

score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=32)

313/313 [==============================] - 5s 7ms/step - loss: 0.3242 - accuracy: 0.8973
